In [45]:
import numpy as np

np.random.seed(1337)  # for reproducibility
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
from sklearn.metrics.classification import accuracy_score

from dbn import SupervisedDBNClassification
# use "from dbn import SupervisedDBNClassification" for computations on CPU with numpy

import pandas as pd

In [46]:
dataset_path = '../datasets/synergy-final-iter1-features/'

device = 'xdk_1'

df = pd.read_pickle(dataset_path + device + '.p')
df_labels = pd.read_pickle(dataset_path + device + '_labels.p')

# filter activities
use_activities = [0,1,2,3,4,5,6,7]
df_labels = df_labels.loc[df_labels.label.isin(use_activities)]
df = df.loc[df.index.isin(df_labels.index)]

df = df.filter(regex='accel_.*index_mass_quantile')

X = df.values
y = df_labels['label'].values

In [47]:
# Splitting data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

X_train

array([[ 583.        ,  583.10204082,  583.20408163, ...,  583.6122449 ,
         583.71428571,  583.81632653],
       [ 531.28571429,  531.3877551 ,  531.48979592, ...,  531.93877551,
         532.02040816,  532.08163265],
       [ 104.56862745,  104.66666667,  104.76470588, ...,  105.1372549 ,
         105.25490196,  105.37254902],
       ..., 
       [ 588.        ,  588.10204082,  588.20408163, ...,  588.55102041,
         588.69387755,  588.79591837],
       [ 195.30612245,  195.40816327,  195.51020408, ...,  195.89795918,
         195.97959184,  196.10204082],
       [ 942.02040816,  942.12244898,  942.2244898 , ...,  942.59183673,
         942.71428571,  942.81632653]])

In [60]:


# Training
classifier = SupervisedDBNClassification(hidden_layers_structure=[8, 8],
                                         learning_rate_rbm=0.1,
                                         learning_rate=0.1,
                                         n_epochs_rbm=10,
                                         n_iter_backprop=100,
                                         batch_size=128,
                                         activation_function='relu',
                                         dropout_p=0.2)
classifier.fit(X_train, y_train)

# Save the model
classifier.save('model.pkl')

[START] Pre-training step:
>> Epoch 1 finished 	RBM Reconstruction error 6582303.791394
>> Epoch 2 finished 	RBM Reconstruction error 6582303.791394
>> Epoch 3 finished 	RBM Reconstruction error 5597292.692576
>> Epoch 4 finished 	RBM Reconstruction error 4257508133150872567808.000000
>> Epoch 5 finished 	RBM Reconstruction error 3724856.189623
>> Epoch 6 finished 	RBM Reconstruction error 2903755.893008
>> Epoch 7 finished 	RBM Reconstruction error 2507208.309773
>> Epoch 8 finished 	RBM Reconstruction error 2327019.034231
>> Epoch 9 finished 	RBM Reconstruction error 2240572.936694
>> Epoch 10 finished 	RBM Reconstruction error 2200576.626904
>> Epoch 1 finished 	RBM Reconstruction error 0.004237
>> Epoch 2 finished 	RBM Reconstruction error 0.001781
>> Epoch 3 finished 	RBM Reconstruction error 0.000694
>> Epoch 4 finished 	RBM Reconstruction error 0.000214
>> Epoch 5 finished 	RBM Reconstruction error 0.000054
>> Epoch 6 finished 	RBM Reconstruction error 0.000013
>> Epoch 7 finish

In [61]:
# Restore it
classifier = SupervisedDBNClassification.load('model.pkl')

# Test
y_pred = classifier.predict(X_test)
print('Done.\nAccuracy: %f' % accuracy_score(y_test, y_pred))

Done.
Accuracy: 0.077586
